### Whisp a GeoJSON via Google drive
- Use this when "whisp_geojson_to_csv' notebook fails due to time out errors
- NB requires collecting outputs from Google Drive 

Setup
- NB use a virtual environment to avoid altering your python environment (https://docs.python.org/3/tutorial/venv.html)

In [ ]:
# Earth Engine and Common Libraries
import ee
from pathlib import Path

# Authenticate and initialize Earth Engine
try:
    ee.Initialize()  # Try to use existing credentials first
except Exception:
    ee.Authenticate()
    ee.Initialize()

In [ ]:
# Install openforis-whisp (if not already installed)
!pip install --pre openforis-whisp

In [ ]:
import openforis_whisp as whisp

Get a feature collection

In [ ]:
GEOJSON_EXAMPLE_FILEPATH = (
    Path.cwd().parents[0]/ "tests" / "fixtures" / "geojson_example.geojson"
)
print(GEOJSON_EXAMPLE_FILEPATH)


Whisp it

In [ ]:
whisp.whisp_stats_geojson_to_drive(GEOJSON_EXAMPLE_FILEPATH)

Manual step 
- download from Google Drive when finished
- place in output folder (or alternative location)



Import CSV of raw stats


In [ ]:
# Define output directory
out_directory = Path.home() / 'Downloads' 

# Define output file path
stats_file_from_drive = out_directory / "whisp_output_table.csv" # edit as required

df_stats = pd.read_csv(stats_file_from_drive)


Display table 

In [ ]:
# raw stats
df_stats

Format stats based on Whisp schema

In [ ]:

df_stats = whisp.convert_iso3_to_iso2(df=df_stats, iso3_column="Country", iso2_column="ProducerCountry") # temp conversion to add iso2 column

df_formatted_stats = whisp.validate_dataframe_using_lookups(df_stats)


Display table
- Note: If this doesn't look right, check previous steps (including if using the correct downloaded csv results)

In [ ]:
df_formatted_stats #view output dataframe


Calculate risk category

In [ ]:
# add risk columns to end of dataframe
df_w_risk = whisp.whisp_risk(df=df_formatted_stats)

Display table with risk columns

In [ ]:
df_w_risk

Export table with risk columns to csv 

In [ ]:
# Define output file path
output_risk_file = out_directory / "whisp_output_table_w_risk.csv" # edit as required

# Save statistics with added risk columns to CSV
df_w_risk.to_csv(path_or_buf=output_risk_file,index=False)

print(f"Table with risk columns saved to: {output_risk_file}")

Export to GeoJSON (optional)

In [ ]:
# Define the output file path for GeoJSON
geojson_output_file = out_directory / 'whisp_output_table.geojson'

# Save the GeoJSON file
whisp.convert_df_to_geojson(df_w_risk, geojson_output_file)  # builds a geojson file containing Whisp columns. Uses the geometry column "geo" to create the spatial features.
print(f"GeoJSON file saved to: {geojson_output_file}")